### DECK GENERATION AND HAND DISTRIBUTION

In [ ]:
def generate_deck()->list:
    # donot hard code this function.
    # S2,S3, ... SA,H2,H3,...,HA,C2,C3,...,CA,D2,D3,...,DA in the same order
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
    suits = ['S', 'H', 'C', 'D']
    deck = []
    for suit in suits:
      for rank in ranks:
        card = suit + rank;
        deck.append(card);
    return deck

cards = generate_deck()
n_hands = 5
import random
random.seed(5)
random.shuffle(cards)
# make sure that generate deck generate the deck in the given order and use the above random seed for shuffling.


def distribute_cards(cards:list,n_hands:int)-> list[list]:
    # cards distributed one by one starting from index 0 going in circles.
    hands = [[] for _ in range(n_hands)]  # Create empty lists for each hand

    # Distribute cards one by one in a circular manner
    current_hand = 0
    for i, card in enumerate(cards):
        hands[current_hand].append(card)  # Add the current card to the current hand
        current_hand = (current_hand + 1) % n_hands  # Move to the next hand in a circular fashion

    return hands

hands = distribute_cards(cards, n_hands)
print(hands)


[['SK', 'C3', 'DJ', 'C7', 'H8', 'C10', 'CK', 'D5', 'S5', 'C9', 'H5'], ['H7', 'DQ', 'HA', 'H6', 'S6', 'S7', 'S2', 'C6', 'D9', 'D4', 'D2'], ['SJ', 'CQ', 'D3', 'H10', 'S4', 'S10', 'CJ', 'HQ', 'H4', 'D7'], ['CA', 'DK', 'C8', 'H3', 'C2', 'H2', 'S8', 'S9', 'C5', 'HJ'], ['D6', 'DA', 'C4', 'H9', 'SA', 'D8', 'HK', 'SQ', 'S3', 'D10']]


### PLAYING A ROUND

In [ ]:
def play_hand(hand: list, cards_on_the_table: list) -> bool:
    if not hand:
        raise ValueError("Hand is empty. Cannot play a card.")

    # Sort the hand to prioritize playing the smallest card of the same suit if possible
    sorted_hand = sorted(hand, key=lambda x: (get_card_value(x), x[-1]))

    # Determine the suit of the starting card on the table (if any)
    starting_suit = cards_on_the_table[0][-1] if cards_on_the_table else None

    # Try to play the smallest card of the same suit as the starting suit (if available)
    for card in sorted_hand:
        if starting_suit and card[-1] == starting_suit:
            hand.remove(card)
            cards_on_the_table.append(card)
            return False  # Not collecting cards in this case

    # If no card of the same suit can be played, play the highest card in any suit
    highest_card = max(sorted_hand, key=lambda x: (get_card_value(x), x[-1]))
    hand.remove(highest_card)
    cards_on_the_table.append(highest_card)
    return True  # Collecting cards in this case

def get_card_value(card: str) -> int:
    rank = card[:-1]
    if rank.isdigit():
        return int(rank)
    elif rank == 'A':
        return 14
    elif rank == 'K':
        return 13
    elif rank == 'Q':
        return 12
    elif rank == 'J':
        return 11

def play_round(hands: list[list], starting_player: int) -> int:
    cards_on_the_table = []
    current_player = starting_player
    while True:
        hand = hands[current_player]
        collecting_cards = play_hand(hand, cards_on_the_table)
        if collecting_cards:
            # Collect cards from the table to current player's hand
            hands[current_player].extend(cards_on_the_table)
            break
        current_player = (current_player + 1) % len(hands)
    return (starting_player + 1) % len(hands)

def n_remaining(hands):
    return sum(map(lambda x: len(x)>0,hands))


### ACTUAL GAME STARTS

In [ ]:
def play_game(hands: list[list]) -> tuple:
    # Find the starting player based on the hand with the "SA" (Ace of Spades) card
    starting_player = None
    for i, hand in enumerate(hands):
        if "SA" in hand:
            starting_player = i
            break
    if starting_player is None:
        raise ValueError("No player has the Ace of Spades (SA) card.")

    current_player = starting_player
    while n_remaining(hands) > 1:
        current_player = play_round(hands, current_player)

    loser = next(i for i, hand in enumerate(hands) if hand)
    return loser, hand

loser, hand = play_game(hands)
print(loser,hand)